# Match most similar competency

## Existing competencies

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
competencys = ['Creativity', 'Employee loyalty', 'Reliability', 'Leadership', 'Passion', 'People Skills', 'Proactivity', 'Team Work']
for i in range(8):
    print "{}:{}".format(i, competencys[i])


0:Creativity
1:Employee loyalty
2:Reliability
3:Leadership
4:Passion
5:People Skills
6:Proactivity
7:Team Work


In [4]:
descriptions = ['Ability to think in a divergent way, and generate novel ideas.',
                'Be committed to the success of the organization and believe that working for this organization is their best option.',
                'Ability to performing to perform when needed, finishing projects and meeting deadlines.', 
                'Ability to engage and facilitate others to create a better result.', 
                'Desire to provide long-term commitment to their organization, demonstrate peak performance, and maintain increased tenure with the organization. With the intense desire or enthusiasm for the work.', 
                'Ability to establish social connections and understand the mindset of others.', 
                'Take the initiative and responsibility in improving business, rather than looking for causes in outside circumstances or other people.',
                'Ability to adapt to the needs of a group of people, while supporting the work of others.']

## load dictionary and corpus

In [5]:
from gensim import corpora, similarities, models

dictionary = corpora.Dictionary.load_from_text('/mnt/local/var/seedlink/data/dictionary/dict_en.txt')
print(dictionary)

Dictionary(50920 unique tokens: [u'sucess', u"cake'", u'mid-week', u'stallions', u'sonja']...)


In [6]:
import codecs
import json
path = '/mnt/local/var/seedlink/data/nltk_data/corpora/prepcorp/en_cc.json'
with codecs.open(path, 'r', 'utf8') as fp:
    jdata = fp.read()
data = json.loads(jdata)
corpus = data

In [7]:
for i in range(10):
    words =  [dictionary[t[0]] for t in corpus[i]]
     #print corpus[i]
    #print ' '.join(words)

## New competency

In [8]:
new_comp = """
Charisma	Able to establish social connections and a broad network
Confidence	Acts on his/her own initiative with confidence
Creativity	Seeks change and performs creatively
Entrepreneurial	Keeps aware of both organizational issues and market opportunities
Goal Orientated	Works systematically and drives the project to its goals
Goal Orientation	Work systematically and drive the project to the target goals
Growth Potential	Growth Potential
Integrity	Acts with integrity and shows social responsibility
Intellect	Flexible application of knowledge to differing contexts
Job Performance	Job Performance
Leadership	Identifies talents, empowers and motivates other
Logic	Logical thinking and rational analysis
Organisation	Sets objectives, balances resources and time, monitors progress
Organization	Sets objectives, balances resources and time, monitors progress
Performance	Job Performance
Proactive	Proactively deals with ambiguity
Process driven	Demonstrate commitment, accountablity, follow policies and procedures of the organizations
Resilient	Coping and resilient under stressful environment
Self motivated	Work enthusiastically for the achievement of self-development
Strategic Competency	Strategic Competency
Strategy	Foresight for potential problems and comes up with appropriate solutions
Teamwork	Adapts to the team; Supports others
"""

lines = new_comp.lower().strip().split('\n')
new_comp_dict = {splitted[0]:splitted[1] for splitted in [line.split('\t') for line in lines]}

## Tfidf

In [9]:
tfidf = models.TfidfModel(corpus)

In [34]:
desc_bow = [dictionary.doc2bow(desc.lower().split()) for desc in descriptions]
desc_tfidf = tfidf[desc_bow]

index = similarities.MatrixSimilarity(desc_tfidf) # transform corpus to LSI space and index itb

In [11]:
# query by competency name
for comp in new_comp_dict:
    query = comp.lower()
    query_bow = dictionary.doc2bow(query.split())
    query_tfidf = tfidf[query_bow]

    sims = index[query_tfidf]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Creativity          	0.0
growth potential         	Creativity          	0.0
integrity                	Creativity          	0.0
intellect                	Creativity          	0.0
confidence               	Creativity          	0.0
goal orientated          	Creativity          	0.0
job performance          	Creativity          	0.0
organisation             	Creativity          	0.0
creativity               	Creativity          	0.0
self motivated           	Creativity          	0.0
strategy                 	Creativity          	0.0
charisma                 	Creativity          	0.0
teamwork                 	Creativity          	0.0
resilient                	Creativity          	0.0
performance              	Creativity          	0.0
goal orientation         	Creativity          	0.0
leadership               	Creativity          	0.0
entrepreneurial          	Creativity          	0.0
logic                    	Creativity          	0.0
organization             	Emplo

In [14]:
# query by competency desc
for comp in new_comp_dict:
    query = new_comp_dict[comp].lower()
    query_bow = dictionary.doc2bow(query.split())
    query_tfidf = tfidf[query_bow]

    sims = index[query_tfidf]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Creativity          	0.0
growth potential         	Creativity          	0.0
integrity                	Proactivity         	0.193758636713
intellect                	Team Work           	0.00204158388078
confidence               	Proactivity         	0.395330578089
goal orientated          	Leadership          	0.00118970533367
job performance          	Creativity          	0.0
organisation             	Creativity          	0.00032098844531
creativity               	Creativity          	0.000327974150423
self motivated           	Team Work           	0.0512905456126
strategy                 	Passion             	0.00524804694578
charisma                 	People Skills       	0.552750945091
teamwork                 	Leadership          	0.118315845728
resilient                	Creativity          	0.000263188820099
performance              	Creativity          	0.0
goal orientation         	Team Work           	0.0381039567292
leadership               	Proactivit

## LSI

In [15]:
lsi = models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=300)

In [16]:
lsi.add_documents(desc_tfidf)

In [17]:
index = similarities.MatrixSimilarity(lsi[desc_tfidf]) # transform corpus to LSI space and index it

In [18]:
# query by competency name
for comp in new_comp_dict:
    query = comp.lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lsi = lsi[tfidf[query_bow]]

    sims = index[query_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Reliability         	0.16094648838
growth potential         	Passion             	0.0644728988409
integrity                	Passion             	0.163791120052
intellect                	Reliability         	0.107082344592
confidence               	Passion             	0.158602669835
goal orientated          	Employee loyalty    	0.187772765756
job performance          	Employee loyalty    	0.0336804315448
organisation             	Team Work           	0.0867025107145
creativity               	People Skills       	0.103887081146
self motivated           	Leadership          	0.0814848914742
strategy                 	Team Work           	0.142499417067
charisma                 	People Skills       	0.134117424488
teamwork                 	Proactivity         	0.0475586280227
resilient                	Team Work           	0.0936880260706
performance              	Employee loyalty    	0.0185290426016
goal orientation         	Employee loyalty    	0.187408268452
le

In [19]:
# query by competency desc
for comp in new_comp_dict:
    query = new_comp_dict[comp].lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lsi = lsi[tfidf[query_bow]]

    sims = index[query_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Reliability         	0.16094648838
growth potential         	Passion             	0.0644728988409
integrity                	Passion             	0.440464019775
intellect                	People Skills       	0.719145953655
confidence               	Passion             	0.232585817575
goal orientated          	People Skills       	0.240231990814
job performance          	Employee loyalty    	0.0336804315448
organisation             	People Skills       	0.304395973682
creativity               	People Skills       	0.32253241539
self motivated           	Team Work           	0.597432136536
strategy                 	Passion             	0.308161914349
charisma                 	People Skills       	0.614012718201
teamwork                 	People Skills       	0.557689547539
resilient                	People Skills       	0.349830627441
performance              	Employee loyalty    	0.0336804315448
goal orientation         	Team Work           	0.617725491524
leaders